In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import schedules, Adam
from tensorflow.keras import callbacks
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Directories
train_dir = "C:/Users/Mahendra/Downloads/htr research paper/DevanagariHandwrittenCharacterDataset/Train"
test_dir = "C:/Users/Mahendra/Downloads/htr research paper/DevanagariHandwrittenCharacterDataset/Test"


In [3]:
# Hyperparameters
img_height = 64
img_width = 64
batch_size = 32
num_classes = 46  # Assuming 46 classes


In [4]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [5]:
# Create training and validation datasets with data augmentation
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    label_mode="categorical",
    image_size=(img_height, img_width),
    batch_size=batch_size
)



Found 78200 files belonging to 46 classes.
Using 62560 files for training.


In [6]:
val_ds = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    label_mode="categorical",
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 78200 files belonging to 46 classes.
Using 15640 files for validation.


In [7]:
# Test dataset
test_ds = image_dataset_from_directory(
    test_dir,
    seed=123,
    label_mode="categorical",
    image_size=(img_height, img_width),
    batch_size=batch_size
)


Found 13800 files belonging to 46 classes.


In [8]:
# AUTOTUNE optimizes dataset loading
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)



In [9]:
model = models.Sequential([
    data_augmentation,  # Apply data augmentation
    
    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.BatchNormalization(),
    layers.SeparableConv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 2
    layers.SeparableConv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Convolutional Layer 3
    layers.SeparableConv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),  # Using Global Average Pooling instead of flattening

    # Dense Layers with Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    
    layers.Dense(num_classes, activation='softmax')  # Output layer
])



In [10]:
# Compile the model with Adam optimizer and learning rate scheduler
initial_learning_rate = 1e-3
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [11]:
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


In [12]:
for images, labels in train_ds.take(1):  # Take one batch from training data
    model(images)  # This "calls" the model with data and builds it

model.summary()  # Now the model is built, and you can print the summary


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 64, 64, 3)           0         
                                                                 
 conv2d (Conv2D)             (32, 62, 62, 32)          896       
                                                                 
 batch_normalization (BatchN  (32, 62, 62, 32)         128       
 ormalization)                                                   
                                                                 
 separable_conv2d (Separable  (32, 60, 60, 64)         2400      
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (32, 60, 60, 64)         256       
 hNormalization)                                                 
                                                      

In [ ]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stopping]
)


Epoch 1/15
 248/1955 [==>...........................] - ETA: 16:13 - loss: 3.7256 - accuracy: 0.0878

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc * 100:.2f}%")

In [ ]:
# Plotting Accuracy and Loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
